In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
def build_lenet_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess MNIST data
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
def train_lenet(model, train_data, train_labels, test_data, test_labels):
    model.save_weights(init_weights)
    model.save(init_model)
    initailsoftmax = model.predict(test_data)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        loss, accuracy = model.evaluate(test_data, test_labels, verbose=0)
        results = model.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)

        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
        if epoch ==25:
          model.save_weights(final_weights)
          model.save(final_model)
        epoch += 1




In [ ]:
model_name = 'LeNet'
dataset = 'MNIST'

# Creating Base Models

In [ ]:
filepath = '/content/base/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset +'_init.tf'
final_weights = filepath + model_name + dataset +'_final_weights.tf'
final_model= filepath + model_name + dataset +'_final.tf'

In [ ]:
!mkdir '/content/base/'

In [ ]:
base = build_lenet_model(input_shape, num_classes)

In [ ]:
base.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
train_lenet(base,x_train,y_train,x_test,y_test)

313/313 [==============================] - 3s 3ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.06029600650072098, Test Accuracy: 0.9812999963760376
Epoch 2:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04760419949889183, Test Accuracy: 0.9854000210762024
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.043411146849393845, Test Accuracy: 0.9868000149726868
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.037612270563840866, Test Accuracy: 0.9886000156402588
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.039779212325811386, Test Accuracy: 0.9883999824523926
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04945624619722366, Test Accuracy: 0.9871000051498413
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.03659588471055031, Test Accuracy: 0.9901000261306763
Epoch 8:
313/313 [==============

In [ ]:
del (base)
gc.collect()
keras.backend.clear_session()

# Independant model one

In [ ]:
filepath = '/content/independant_1/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + '_final.tf'

In [ ]:
!mkdir  '/content/independant_1/'

In [ ]:
model_independant_1 = build_lenet_model(input_shape, num_classes)

In [ ]:
train_lenet(model_independant_1,x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.05130818486213684, Test Accuracy: 0.9829999804496765
Epoch 2:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04899435490369797, Test Accuracy: 0.9836999773979187
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04341496154665947, Test Accuracy: 0.9865000247955322
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04148406162858009, Test Accuracy: 0.9872999787330627
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.03782878443598747, Test Accuracy: 0.9894000291824341
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.03661338612437248, Test Accuracy: 0.9904000163078308
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.03982030600309372, Test Accuracy: 0.9884999990463257
Epoch 8:
313/313 [=================

In [ ]:
del (model_independant_1)
gc.collect()
keras.backend.clear_session()

# Independant 2

In [ ]:
filepath = '/content/independant_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights =filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + '_final.tf'

In [ ]:
!mkdir '/content/independant_2/'

In [ ]:
model_independant_2 = build_lenet_model(input_shape, num_classes)

In [ ]:
train_lenet(model_independant_2,x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.061435822397470474, Test Accuracy: 0.9801999926567078
Epoch 2:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.051581598818302155, Test Accuracy: 0.982699990272522
Epoch 3:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.0456363782286644, Test Accuracy: 0.9851999878883362
Epoch 4:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.039891958236694336, Test Accuracy: 0.9876999855041504
Epoch 5:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.032890237867832184, Test Accuracy: 0.9908000230789185
Epoch 6:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04208258166909218, Test Accuracy: 0.9886999726295471
Epoch 7:
313/313 [==============================] - 1s 2ms/step
Test Loss: 0.04225752502679825, Test Accuracy: 0.9884999990463257
Epoch 8:
313/313 [===============

In [ ]:
del (model_independant_2)
gc.collect()
keras.backend.clear_session()

#Download Independant Models

In [ ]:
shutil.make_archive("base", 'zip', "base")
files.download("base.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("independant_1", 'zip', "independant_1")
files.download("independant_1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

shutil.make_archive("independant_2", 'zip', "independant_2")
files.download("independant_2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1

# TO DO EVALUATE FUNCTION


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

# Student 0.1 Alpha

In [ ]:
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)


In [ ]:
teacher_model_path = '/content/base/LeNetMNIST_final.tf'
techer_model = load_model(teacher_model_path)

In [ ]:
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)

In [ ]:
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

In [ ]:
!mkdir '/content/student_same/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 0.0741 - accuracy: 0.9777
Test Loss: 0.07406283915042877, Test Accuracy: 0.9776999950408936
Epoch 2:
313/313 [==============================] - 1s 2ms/step - loss: 0.0553 - accuracy: 0.9828
Test Loss: 0.05526040121912956, Test Accuracy: 0.9828000068664551
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 0.0485 - accuracy: 0.9840
Test Loss: 0.048512086272239685, Test Accuracy: 0.984000027179718
Epoch 4:
313/313 [==============================] - 1s 2ms/step - loss: 0.0442 - accuracy: 0.9859
Test Loss: 0.04421151429414749, Test Accuracy: 0.9858999848365784
Epoch 5:
313/313 [==============================] - 1s 4ms/step - loss: 0.0400 - accuracy: 0.9869
Test Loss: 0.039986465126276016, Test Accuracy: 0.9868999719619751
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.0381 - accuracy: 0.9873
Test Loss: 0.038060590624809265, Test Accuracy: 0.9872999787330627
Epoch 7:
313/313 [=====

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

# Student same Alpha 0.5

In [ ]:
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

In [ ]:
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

In [ ]:
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

In [ ]:
!mkdir '/content/student_same_50/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 3ms/step - loss: 0.0749 - accuracy: 0.9769
Test Loss: 0.07488416135311127, Test Accuracy: 0.9768999814987183
Epoch 2:
313/313 [==============================] - 1s 3ms/step - loss: 0.0564 - accuracy: 0.9821
Test Loss: 0.05644461140036583, Test Accuracy: 0.9821000099182129
Epoch 3:
313/313 [==============================] - 1s 4ms/step - loss: 0.0497 - accuracy: 0.9841
Test Loss: 0.049728453159332275, Test Accuracy: 0.9840999841690063
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0450 - accuracy: 0.9850
Test Loss: 0.04503950849175453, Test Accuracy: 0.9850000143051147
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0404 - accuracy: 0.9870
Test Loss: 0.04039209708571434, Test Accuracy: 0.9869999885559082
Epoch 6:
313/313 [==============================] - 1s 3ms/step - loss: 0.0383 - accuracy: 0.9869
Test Loss: 0.038291968405246735, Test Accuracy: 0.9868999719619751
Epoch 7:
313/313 [=====

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

# Student same Alpha 0.9

In [ ]:
model_path = '/content/base/LeNetMNIST_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'

In [ ]:
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 1s 2ms/step
Epoch 1:


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5575: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 [==============================] - 1s 4ms/step - loss: 0.0750 - accuracy: 0.9770
Test Loss: 0.07502587139606476, Test Accuracy: 0.9769999980926514
Epoch 2:
313/313 [==============================] - 1s 2ms/step - loss: 0.0547 - accuracy: 0.9826
Test Loss: 0.05474058911204338, Test Accuracy: 0.9825999736785889
Epoch 3:
313/313 [==============================] - 1s 3ms/step - loss: 0.0476 - accuracy: 0.9842
Test Loss: 0.047644395381212234, Test Accuracy: 0.9842000007629395
Epoch 4:
313/313 [==============================] - 1s 3ms/step - loss: 0.0434 - accuracy: 0.9862
Test Loss: 0.04341650381684303, Test Accuracy: 0.9861999750137329
Epoch 5:
313/313 [==============================] - 1s 3ms/step - loss: 0.0408 - accuracy: 0.9863
Test Loss: 0.04079493507742882, Test Accuracy: 0.986299991607666
Epoch 6:
313/313 [==============================] - 2s 5ms/step - loss: 0.0375 - accuracy: 0.9878
Test Loss: 0.037476472556591034, Test Accuracy: 0.9878000020980835
Epoch 7:
313/313 [======

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()

---

# Download Distill Data

In [ ]:
shutil.make_archive("student_same", 'zip', "student_same")
files.download("student_same.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("student_same_50", 'zip', "student_same_50")
files.download("student_same_50.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("student_same_90", 'zip', "student_same_90")
files.download("student_same_90.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---

# Qunatisation

https://github.com/sonalimedani/TF_Quantization/blob/master/quantization.ipynb

https://medium.com/@sonalimedani/post-training-quantization-with-tensorflow-lite-on-a-keras-model-f373068966c4

**Default**: 8 Bit

In [ ]:
path = '/content/base/LeNetMNIST_final.tf'
model = tf.keras.models.load_model(path)
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0454 - accuracy: 0.9916


[0.04537806659936905, 0.991599977016449]

In [ ]:
filepath = '/content/quantised_8_bit/'
!mkdir '/content/quantised_8_bit/'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open(filepath+"8_bit.tflite", "wb").write(tflite_quant_model)

230912

In [ ]:
import os
print("Float model in Mb:", os.path.getsize(filepath+"8_bit.tflite") / float(2**20))
print("Quantized model in Mb:", os.path.getsize(filepath+"8_bit.tflite") / float(2**20))
print("Compression ratio:", os.path.getsize(filepath+"8_bit.tflite")/os.path.getsize(path))

Float model in Mb: 0.22021484375
Quantized model in Mb: 0.22021484375
Compression ratio: 56.375


In [ ]:
interpreter = tf.lite.Interpreter(model_path=(filepath+"8_bit.tflite"))
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_default = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_default = pd.concat([results_default, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

99.18


In [ ]:
results_default

,0,1,2,3,4,5,6,7,8,9
0,1.177629e-19,1.461635e-20,5.603971e-17,9.389258e-16,1.315932e-19,4.092707e-22,2.931911e-27,1.000000e+00,1.169599e-18,1.184309e-14
1,1.749664e-18,1.458822e-21,1.000000e+00,4.565912e-25,4.101402e-27,0.000000e+00,2.972112e-20,2.367044e-28,2.212697e-23,4.530275e-33
2,5.831785e-18,1.000000e+00,2.675095e-17,3.456411e-22,1.270661e-10,2.398584e-14,1.321247e-15,1.255180e-13,7.771570e-15,3.255722e-19
3,1.000000e+00,2.613684e-25,2.115442e-22,2.566134e-18,2.020771e-24,5.159869e-15,1.544984e-09,2.171396e-14,2.548142e-21,4.397766e-18
4,8.178968e-18,7.137039e-17,3.638298e-16,1.673180e-23,9.999937e-01,8.187830e-19,4.630382e-16,6.117955e-21,5.829082e-17,6.365745e-06
...,...,...,...,...,...,...,...,...,...,...
9995,4.070052e-23,5.753127e-13,1.000000e+00,4.482278e-19,2.699927e-23,3.907887e-34,2.291808e-24,3.073104e-15,6.166666e-19,1.575542e-26
9996,4.319460e-31,3.846011e-18,8.823528e-22,1.000000e+00,1.133233e-25,4.138565e-15,6.690279e-34,1.163051e-23,8.122469e-24,2.371891e-19
9997,1.606666e-33,1.111614e-20,5.366531e-26,1.035240e-31,1.000000e+00,3.095992e-22,1.388680e-23,3.930089e-26,6.708530e-18,3.236684e-22
9998,6.764088e-25,5.856127e-26,8.921367e-31,1.183587e-20,1.687457e-32,1.000000e+00,9.694434e-17,1.347924e-29,1.065943e-12,6.736749e-30


In [ ]:
results_default.to_csv((filepath+"25_softmax.csv"),index=False)

**16 bit**

In [ ]:
model = tf.keras.models.load_model(path)


In [ ]:
filepath = '/content/quantised_16_bit/'
!mkdir '/content/quantised_16_bit/'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model = converter.convert()
#saving converted model in "converted_quant_model.tflite" file
open((filepath +"16_bit.tflite"), "wb").write(tflite_quant_model)

454608

In [ ]:
import os
print("Float model in Mb:", os.path.getsize(filepath +"16_bit.tflite") / float(2**20))
print("Quantized model in Mb:", os.path.getsize(filepath +"16_bit.tflite") / float(2**20))
print("Compression ratio:", os.path.getsize(filepath +"16_bit.tflite")/os.path.getsize(path))

Float model in Mb: 0.4335479736328125
Quantized model in Mb: 0.4335479736328125
Compression ratio: 110.98828125


In [ ]:
interpreter = tf.lite.Interpreter(model_path=filepath +"16_bit.tflite")
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
results_16_b = pd.DataFrame()
for i in range(len(x_test)):
    input_data = x_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    row = pd.DataFrame(output_data)
    results_16_b = pd.concat([results_16_b, row], ignore_index=True)
    if(np.argmax(output_data) == np.argmax(y_test[i])):
        acc+=1
acc = acc/len(x_test)
print(acc*100)

99.16


In [ ]:
results_16_b

,0,1,2,3,4,5,6,7,8,9
0,9.170932e-20,1.300497e-20,5.098796e-17,8.854204e-16,1.171411e-19,3.634967e-22,2.565476e-27,1.000000e+00,1.062100e-18,9.754578e-15
1,1.626350e-18,1.572880e-21,1.000000e+00,4.067006e-25,3.697767e-27,0.000000e+00,2.759260e-20,2.436441e-28,1.982254e-23,4.451643e-33
2,5.153350e-18,1.000000e+00,2.883522e-17,3.531958e-22,1.305367e-10,2.503961e-14,1.339047e-15,1.266200e-13,8.012639e-15,3.188321e-19
3,1.000000e+00,2.145157e-25,1.868453e-22,2.603692e-18,1.965956e-24,5.672306e-15,1.496271e-09,2.150954e-14,2.721168e-21,4.523402e-18
4,8.345373e-18,8.471272e-17,3.770911e-16,1.993007e-23,9.999933e-01,8.904704e-19,4.555395e-16,7.522570e-21,6.255766e-17,6.660656e-06
...,...,...,...,...,...,...,...,...,...,...
9995,3.026221e-23,5.194900e-13,1.000000e+00,3.779254e-19,2.090804e-23,3.367062e-34,1.928917e-24,2.812551e-15,5.057124e-19,1.129586e-26
9996,3.152575e-31,3.132133e-18,7.223371e-22,1.000000e+00,8.972072e-26,3.191466e-15,4.365188e-34,7.712321e-24,6.485983e-24,1.923961e-19
9997,1.206960e-33,9.945840e-21,4.750222e-26,9.760114e-32,1.000000e+00,2.914390e-22,1.006986e-23,3.449213e-26,6.106252e-18,3.057996e-22
9998,6.442432e-25,4.627774e-26,6.963039e-31,9.373846e-21,1.355331e-32,1.000000e+00,9.423952e-17,1.069579e-29,9.256585e-13,5.974306e-30


In [ ]:
results_16_b.to_csv(filepath +'25_softmax.csv',index =False)

# Download Data

In [ ]:
shutil.make_archive("quantised_16_bit", 'zip', "quantised_16_bit")
files.download("quantised_16_bit.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
shutil.make_archive("quantised_8_bit", 'zip', "quantised_8_bit")
files.download("quantised_8_bit.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>